# <center>Data Mining project: Discover and describe areas of interest<br> and events from geo-located parsed_data</center>

# 0/ Import Dataset and Libraries

## #1 Importing Libraries

- ### Installs

>requires: `pip install ipykernel`

In [ ]:

# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
# plotting 
! pip install matplotlib==3.8.0 
# Language processing
! pip install nltk
#! python -m nltk.downloader popular # popular functions
# Folium
! pip install folium==0.12.1


- ###   Imports

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.neighbors import NearestNeighbors
import sklearn.cluster as cl
import folium
from folium.plugins import MarkerCluster
import random
import numpy as np
import re
from nltk.corpus import stopwords


## #2 Importing Data

In [ ]:
# Charger les données avec low_memory=False pour éviter les avertissements
DATA = pd.read_csv("data/flickr_data2.csv", sep=",", low_memory=False)
random.seed(42)

# I/ Understanding the Data

--Write something to describe this part of the report--

## #1 Data clearing and preparation

In [ ]:
# Print details about parsing step:
# -> Number of lines parsed - % of original data parsed
def parse_conclusion(parsed_data):
    l = len(parsed_data)
    print(f"<Lines parsed: {l} - {round(100*l/len(DATA),3)}% of original data>")

- Étape 1 : Nettoyage initial des colonnes

In [ ]:
# Supprimer les espaces supplémentaires dans les noms des colonnes
parsed_data = DATA
parsed_data.columns = parsed_data.columns.str.strip()

# Convertir les colonnes temporelles en numériques
# Liste des colonnes temporelles
time_columns = [
    'date_taken_minute', 'date_taken_hour', 'date_taken_day',
    'date_taken_month', 'date_taken_year',
    'date_upload_minute', 'date_upload_hour', 'date_upload_day',
    'date_upload_month', 'date_upload_year'
]

# Convertir chaque colonne en int64, remplacer les erreurs par 0
for col in time_columns:
    parsed_data[col] = pd.to_numeric(DATA[col], errors='coerce').fillna(0).astype('int64')

- Étape 2 : Suppression des doublons basés sur l'identifiant unique

In [ ]:
# Sauvegarder les doublons pour audit futur
duplicate_data = parsed_data[parsed_data['id'].duplicated(keep='first')].sort_values("id")
duplicate_data.to_csv(
    "data/parsed_lines/duplicatedId.csv", index=False
)

# Supprimer les doublons
parsed_data = parsed_data[~parsed_data['id'].duplicated(keep='first')]

parse_conclusion(duplicate_data)

- Étape 3 : Gestion des colonnes inutilisées ou corrompues

In [ ]:
# Identifier les colonnes inutiles
unused_columns = ["Unnamed: 16", "Unnamed: 17", "Unnamed: 18", 
                  "date_upload_minute", "date_upload_hour", "date_upload_day", 
                  "date_upload_month", "date_upload_year"]

# Sauvegarder les données corrompues
corrupted_data = parsed_data[parsed_data["Unnamed: 16"].notnull() | parsed_data["Unnamed: 17"].notnull() | parsed_data["Unnamed: 18"].notnull()]
corrupted_data.to_csv(
    "data/parsed_lines/corrupted_data.csv"
, index=False)


# Supprimer les colonnes inutilisées et les lignes corrompues
parsed_data = parsed_data[~(parsed_data["Unnamed: 16"].notnull() | parsed_data["Unnamed: 17"].notnull() | parsed_data["Unnamed: 18"].notnull())]
parsed_data = parsed_data.drop(columns=unused_columns)

parse_conclusion(corrupted_data)

- Étape 4 : Nettoyage des coordonnées GPS exterieures à Lyon

In [ ]:
# Définir les limites géographiques de Lyon
lyon_lat_min, lyon_lat_max = 45.69, 45.85
lyon_lon_min, lyon_lon_max = 4.78, 4.92

# Sauvegarder les données not Lyonnaises ou non definies
out_lyon_data = parsed_data[
    ~((parsed_data['lat'] >= lyon_lat_min) & 
    (parsed_data['lat'] <= lyon_lat_max) &
    (parsed_data['long'] >= lyon_lon_min) &
    (parsed_data['long'] <= lyon_lon_max))
]
out_lyon_data.to_csv(
    "data/parsed_lines/out_lyon.csv"
, index=False)

# Filtrer les données pour garder uniquement les points dans Lyon
parsed_data = parsed_data[
    (parsed_data['lat'] >= lyon_lat_min) & 
    (parsed_data['lat'] <= lyon_lat_max) &
    (parsed_data['long'] >= lyon_lon_min) &
    (parsed_data['long'] <= lyon_lon_max)
]

parse_conclusion(out_lyon_data)

- Étape 5 : Gestion des valeurs manquantes

In [ ]:
# Supprimer les lignes avec des valeurs manquantes
#parsed_data = parsed_data.dropna(subset=['id', 'user', 'lat', 'long', 'tags', 'title', 'date_taken_minute', 'date_taken_hour', 'date_taken_day', 'date_taken_month', 'date_taken_year'])

# Sauvegarder les lignes à données manquantes
val_manquante_data = parsed_data[parsed_data.drop(columns=['tags','title']).isna().any(axis = 1)]
val_manquante_data.to_csv(
    "data/parsed_lines/NaNs.csv"
, index=False)

# Remplir les valeurs manquantes pour les colonnes textuelles par des chaînes vides
parsed_data['tags'] = parsed_data['tags'].fillna('')
parsed_data['title'] = parsed_data['title'].fillna('')

parse_conclusion(val_manquante_data)

- Étape 6 : Filtrage des dates incohérentes

In [ ]:
min_year = 2009

# Sauvegarder les données ayant des date non correcte
uncorrect_date_data = parsed_data[
    ~((parsed_data['date_taken_year'] >= min_year) & (parsed_data['date_taken_year'] <= 2025) &
    (parsed_data['date_taken_month'] >= 1) & (parsed_data['date_taken_month'] <= 12) &
    (parsed_data['date_taken_day'] >= 1) & (parsed_data['date_taken_day'] <= 31) &
    (parsed_data['date_taken_hour'] >= 0) & (parsed_data['date_taken_hour'] <= 23) &
    (parsed_data['date_taken_minute'] >= 0) & (parsed_data['date_taken_minute'] <= 59))
]
uncorrect_date_data.to_csv(
    "data/parsed_lines/uncorrect_date.csv"
, index=False)

# Garder uniquement les dates raisonnables (entre 2009 et 2025)
parsed_data = parsed_data[
    (parsed_data['date_taken_year'] >= min_year) & (parsed_data['date_taken_year'] <= 2025) &
    (parsed_data['date_taken_month'] >= 1) & (parsed_data['date_taken_month'] <= 12) &
    (parsed_data['date_taken_day'] >= 1) & (parsed_data['date_taken_day'] <= 31) &
    (parsed_data['date_taken_hour'] >= 0) & (parsed_data['date_taken_hour'] <= 23) &
    (parsed_data['date_taken_minute'] >= 0) & (parsed_data['date_taken_minute'] <= 59) 
]

parse_conclusion(uncorrect_date_data)

- Étape 7 : Nettoyage/Standarisation des colonnes textuelles

In [ ]:
# Fonction pour nettoyer le texte
def clean_text(text):
    # Supprimer les caractères spéciaux et passer en minuscules
    return re.sub(r'[^a-zA-Z0-9, ]', '', text).lower()

# Appliquer le nettoyage sur les colonnes textuelles
parsed_data['tags'] = parsed_data['tags'].apply(clean_text)
parsed_data['title'] = parsed_data['title'].apply(clean_text)

# Vérifier les résultats
# parsed_data.head()

- Étape 8 : Ajout de la colonne temps_diff (sec). <br><br> Correspond à la différence en secondes entre l'année 2005 pile (0s,0min,...) et la date de prise de la photo. <br>Utile pour le clustering spatio-temporel.

In [ ]:
print("minimum date: " + str(parsed_data['date_taken_year'].min())) 

# Create a datetime column from the given columns
dates = pd.to_datetime(
    {
        'year': parsed_data['date_taken_year'],
        'month': parsed_data['date_taken_month'],
        'day': parsed_data['date_taken_day']
    }
) + pd.to_timedelta(parsed_data['date_taken_minute'], unit='m') + pd.to_timedelta(parsed_data['date_taken_hour'], unit='h')

# Calculate difference from 1970-01-01 in seconds
parsed_data['temps_diff'] = (dates - pd.Timestamp('2005-01-01')).dt.total_seconds()

# Vérifier les résultats
# parsed_data.head()

- Étape 9 : Standardization.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Appliquer la standardisation
scaler = StandardScaler()
parsed_data[["lat_std","long_std","temps_diff"]] = scaler.fit_transform(parsed_data[["lat", "long","temps_diff"]])

# Vérifier les résultats
# parsed_data.head()

- Résultat final

In [ ]:
# Résumé final des données nettoyées
print("Données nettoyées :")

# Réindexer les données
parsed_data = parsed_data.reset_index(drop=True)


# Sauvegarder les données nettoyées pour les prochaines étapes
parsed_data.to_csv("data/cleaned_flickr_data.csv", index=False)

parsed_data.head(5)

## #2 Visualize Data on map

In [ ]:
# Define the center of Lyon
lyon_lat, lyon_lon = 45.75, 4.85  # Approximate coordinates for Lyon, France

# Create a Folium map
map_lyon = folium.Map(location=[lyon_lat, lyon_lon], zoom_start=12)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(map_lyon)

# Add points from the dataset
for _, row in parsed_data.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],  # Replace with 'lat' and 'long' if necessary
        popup=f"Tags: {row['tags']}, Title: {row['title']}",  # Optional popup information
    ).add_to(marker_cluster)

# Display the map
map_lyon

# Optional: Save the map as an HTML file
map_lyon.save("lyon_map_unclustered.html")

## #3 Clustering data

To cluster the geo-located data from Lyon, we will apply multiple clustering techniques to identify areas of interest and patterns of activity.

Our aim is to :

- Identify dense regions of photo-taking activity, which may correspond to points of interest (e.g., landmarks, parks).
- Highlight outliers or isolated points that may not belong to any significant cluster.
- Compare different clustering techniques to determine the most effective one for this dataset.

### Setup:

In [170]:
clustered_data = parsed_data
features_base = ["lat", "long"]
features_spatio_temp = ["lat_std", "long_std", "temps_diff"]
features_lat_lng = ["lat_std", "long_std"]
lyon_lat, lyon_lon = 45.75, 4.85

In [ ]:
def create_folium_map(clusteredData, clusterInfoColumnName, sampleSize):
    map = folium.Map([lyon_lat, lyon_lon], zoom_start = 12, scrollWheelZoom = True) 
    icon_color = ["#" + ''.join([random.choice('ABCDEF0123456789') for i in range(6)]) for i in range(clusteredData[clusterInfoColumnName].nunique())]
    for i,row in clusteredData.sample(sampleSize).iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['long']],
            radius=5,
            color=icon_color[int(row[clusterInfoColumnName]) - 1],
            fill_color=icon_color[int(row[clusterInfoColumnName]) - 1],
            fill=True,
            popup=row['title']
        ).add_to(map)

    map.save("data/Maps/"+clusterInfoColumnName+"_map.html")

### a) K-means

>Why? Suitable for finding spherical clusters in data.<br>
Input required: Number of clusters (k).<br>
Steps: Apply K-Means with varying values of k to find the optimal number of clusters using the elbow method.


- Finding K with the elbow method

In [ ]:
ks_df = pd.DataFrame(columns=["k", "WCSS"])
for k in range(1,21):
    kmeans = cl.KMeans(n_clusters=k, init='k-means++',random_state=42)
    kmeans.fit(clustered_data[features_lat_lng])
    print("k = " + str(k) + " - WCSS = " + str(kmeans.inertia_))
    new_row = pd.DataFrame({"k": [k], "WCSS": [kmeans.inertia_]})
    ks_df = pd.concat([ks_df, new_row], ignore_index=True)

plt.xlabel("k")
plt.ylabel("WCSS")
plt.plot(ks_df["k"], ks_df["WCSS"], "-xb")

We observe that the best k from the elbow method is k = 5

- Clustering data

In [ ]:
# number of clusters 
k = 5 # set to optimal k found with elbow method
# create a model
kmeans = cl.KMeans(n_clusters=k, init='k-means++',random_state=42)
# fit scaled data
kmeans.fit(clustered_data[features_lat_lng])
# Add labels to data
K_mean_labels = kmeans.labels_
clustered_data = clustered_data.join(pd.DataFrame({"cluster_kmeans": K_mean_labels}))
clustered_data = clustered_data[clustered_data.cluster_kmeans != -1] # Takeout outliers

- Visualizing results

In [ ]:
# Evaluate clustering
inertia = kmeans.inertia_
print(f"Sum of squared distances: {kmeans.inertia_}")

# sample_silhouette_values = silhouette_samples(clustered_data[features_base], K_mean_labels)
# means_lst = []
# for label in range(k):
#     means_lst.append(sample_silhouette_values[K_mean_labels == label].mean())

# Visualisation des clusters sur une carte simple
plt.figure(figsize=(10, 8))
plt.scatter(clustered_data['long'], clustered_data['lat'], c=clustered_data['cluster_kmeans'], cmap='tab10', s=10)
plt.title(f"K-Means Clustering (k={k})")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.colorbar(label="Cluster")
plt.grid()
plt.show()


In [ ]:
# Creation de la Map
create_folium_map(clustered_data, 'cluster_kmeans', 10000)

We can clearly see that the elbow method is insufficient to determine an optimal k given proximity.<br>
K means is also unoptimal given non globular clusters and proximity to each other.

### b) DBSCAN

- Finding Best EPS

In [ ]:
# Convertir les coordonnées de latitude et longitude en radians
# clustered_data["lat_rad"] = np.radians(clustered_data["lat"])
# clustered_data["long_rad"] = np.radians(clustered_data["long"])
# features_rad = ["lat_rad", "long_rad"]

# 2. Recherche du meilleur paramètre `eps` avec la méthode du coude pour les k-plus proches voisins
# Trouver les distances au k-plus proche voisin (on prend k = 5)
nbrs = NearestNeighbors(n_neighbors=5, metric="haversine").fit(clustered_data[features_lat_lng])
distances, indices = nbrs.kneighbors(clustered_data[features_lat_lng])

# Trier les distances pour identifier un coude
sorted_distances = np.sort(distances[:, -1])  # Distance au 5e plus proche voisin
plt.figure(figsize=(8, 5))
plt.plot(sorted_distances)
plt.title("Méthode du Coude pour DBSCAN (Haversine, eps)")
plt.xlabel("Points de Données")
plt.ylabel("Distance au 5e Plus Proche Voisin (en radians)")
plt.grid()
plt.show()

La distance eps optimale est proche de 0.00005 rad. <br>
Or par l'expérimentation nous avons trouvé qu'elle est de 0.0000487.

La distance eps optimale est proche de 0.0125 rad. <br>
Or par l'expérimentation nous avons trouvé qu'elle est de 0.012175.

- Clustering Data

In [ ]:
# 3. Définir un `eps` en fonction du coude observé (par exemple, eps=0.01)
# Note : eps est en radians, correspondant à une distance angulaire (~6371 km * radians pour une distance en km)

# eps = 300 / 6371000
# eps = 0.0051
# min_samples = 70
clustered_data = parsed_data
eps = 0.0051
min_samples = 60

# Appliquer DBSCAN avec la métrique Haversine
dbscan = cl.DBSCAN(eps=eps, min_samples=min_samples, metric="haversine")
dbscan_labels = dbscan.fit_predict(clustered_data[features_lat_lng]) 

In [175]:
# 4. Ajouter les étiquettes aux données
clustered_data["cluster_DBSCAN"] = dbscan_labels

# Compter le nombre de clusters (ignorer les outliers marqués comme -1)
num_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
num_outliers = sum(dbscan_labels == -1)
clustered_data = clustered_data[clustered_data.cluster_DBSCAN != -1] # Takeout outliers

- Visualizing results

Nombre de clusters détectés : 244
Nombre d'outliers : 74477


In [ ]:
# 5. Visualisation des clusters
print(f"Nombre de clusters détectés : {num_clusters}")
print(f"Nombre d'outliers : {num_outliers}")

plt.figure(figsize=(10, 8))
plt.scatter(
    clustered_data['long'], 
    clustered_data['lat'], 
    c=clustered_data['cluster_DBSCAN'], 
    cmap='tab10', 
    s=10
)
plt.title(f"DBSCAN Clustering avec Haversine (eps={eps}, min_samples={min_samples})")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.colorbar(label="Cluster")
plt.grid()
plt.show()

create_folium_map(clustered_data, 'cluster_DBSCAN', 10000)


### c) Hierarchical Clustering

>Why? Provides a dendrogram to visualize the hierarchy of clusters.<br>
Input required: No predefined number of clusters; can be adjusted by cutting the dendrogram.<br>
Steps: Compute a linkage matrix using ward or other linkage methods. Cut the dendrogram to form clusters.

In [ ]:
# 1. Préparation des données
# Sous-échantillonnage des données (garder n points aléatoires pour éviter les blocages)

geo_data_sampled = parsed_data.sample(n=50000, random_state=42)
geo_data_sampled.reset_index(drop=True)

# 2. Création de la matrice de linkage avec les données échantillonnées
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt

linkage_matrix = linkage(geo_data_sampled[features_base], method='ward')

# 3. Visualisation du dendrogramme
plt.figure(figsize=(12, 8))
dendrogram(linkage_matrix, truncate_mode='level', p=5)  # Truncate pour simplifier la visualisation
plt.title("Dendrogramme (Clustering Hiérarchique - Échantillonné)")
plt.xlabel("Points de données")
plt.ylabel("Distance")
plt.show()

# 4. Découpage du dendrogramme pour obtenir les clusters
num_clusters = 100
sampled_clusters = fcluster(linkage_matrix, t=num_clusters, criterion='maxclust')

# Ajouter les clusters à l'échantillon pour analyse
geo_data_sampled['hierarchical_cluster'] = sampled_clusters

- Visualizing results

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(geo_data_sampled['long'], geo_data_sampled['lat'], c=geo_data_sampled['hierarchical_cluster'], cmap='tab10', s=50)
plt.title(f"Clustering Hiérarchique (Nombre de clusters: {num_clusters})")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid()
plt.show()
create_folium_map(geo_data_sampled, 'hierarchical_cluster', 30000)

## #4 post-processing

In [ ]:
post_processed_data = clustered_data
post_processed_data.head(1)

### a) Filter groups where there isn't enough users: low number of samples or very few users withing the cluster.

In [ ]:
user_limit = 5
for cluster_i in range(post_processed_data["cluster_DBSCAN"].nunique()):
    cluster = post_processed_data[post_processed_data.cluster_DBSCAN == cluster_i]
    nbUsers = cluster["user"].nunique()
    if(nbUsers < user_limit):
        cluster.to_csv(
            "data/post_processing/insufficient_users_clusters/"+str(cluster_i)+".csv",
        index=False)
        post_processed_data = post_processed_data[~post_processed_data["id"].isin(cluster["id"])]
parse_conclusion(post_processed_data)

### b) Suppression des stopwords

In [ ]:
# Charger les stopwords en français et en anglais
stop_words = set(stopwords.words('french') + stopwords.words('english'))

def clean_title(raw_title):
    """
    Nettoyer les titres en supprimant les caractères spéciaux sauf les chiffres,
    en normalisant en minuscules, et en supprimant les stopwords.
    """
    # 1. Supprimer les caractères spéciaux sauf les chiffres et normaliser en minuscules
    raw_title = re.sub(r'[^a-zA-Z0-9 ]', '', raw_title.lower())
    
    # 2. Supprimer les stopwords
    meaningful_words = [word for word in raw_title.split() if word not in stop_words]
    
    # 3. Joindre les mots nettoyés en une chaîne
    return " ".join(meaningful_words)

# Appliquer la fonction à la colonne "title"
clustered_data["title_clean"] = clustered_data["title"].fillna("").apply(clean_title)

# Afficher un aperçu des résultats
print(clustered_data[["title", "title_clean"]].head(10))

In [ ]:
def split_tags(raw_tags):
    """
    Diviser les tags par des virgules.
    """
    return raw_tags.split(",") if isinstance(raw_tags, str) else []
    
# Appliquer le split sur la colonne "tags"
clustered_data["tags_split"] = clustered_data["tags"].fillna("").apply(split_tags)

# Afficher un aperçu des tags splittés
print(clustered_data[["tags", "tags_split"]].head(10))